Exploring the participants and the amount of data we have in different databases in Mongodb

comparing the data volume for backup_frigg1 and backup participants

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import os
import sys
import numpy as np
np.seterr(divide = 'ignore') 
import pytz
import time
from datetime import datetime
from functools import reduce
from tqdm import tqdm
tqdm.pandas()

In [2]:
dataPath1 = "/csv/backup_frigg1"
dataPath2 = "/csv/backup"
featurePath = "/csv/features"

backup_frigg1 lockstate data

Lock_state data

In [3]:
dataFilename = "Lock_state.csv"
screenstate = pd.read_csv(os.path.join(dataPath1, dataFilename), sep="|", header=None)
header_list = ["id", "participantId", "attribute", "lck", "timestamp", "uploadtimestamp"]
screenstate.columns = header_list

#change time to Halifax time
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=True)
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"]).dt.tz_convert(tz='America/Halifax')
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=False)

#add new columns to help extract features
screenstate["date"] = screenstate["timestamp"].dt.date
screenstate["time"] = screenstate["timestamp"].dt.strftime('%H:%M:%S')

# keep only Covid participantIds
screenstate = screenstate[screenstate["participantId"].str.contains("PROSITC")]

# print(screenstate.info())

dfa1 = screenstate.groupby("participantId")["date"].nunique().reset_index(name="noOfDates")
dfa2 = screenstate.groupby("participantId")["date"].agg(["min", "max", "count"]).reset_index().rename(columns={'min': 'minDate', 'max': 'maxDate', 'count':'totalCount'})
dfa3 = pd.merge(dfa1, dfa2, on="participantId", how="inner")

# participantIds with more than 14days of data
dfa3 = dfa3[dfa3["noOfDates"]>=14].reset_index(drop=True)

# dfa3

backup lockstate data

In [4]:
dataFilename = "Lock_state.csv"
screenstate = pd.read_csv(os.path.join(dataPath2, dataFilename), sep="|", header=None)
header_list = ["id", "participantId", "attribute", "lck", "timestamp", "uploadtimestamp"]
screenstate.columns = header_list

#change time to Halifax time
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=True)
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"]).dt.tz_convert(tz='America/Halifax')
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=False)

#add new columns to help extract features
screenstate["date"] = screenstate["timestamp"].dt.date
screenstate["time"] = screenstate["timestamp"].dt.strftime('%H:%M:%S')

# keep only Covid participantIds
screenstate = screenstate[screenstate["participantId"].str.contains("PROSITC")]

# print(screenstate.info())

dfb1 = screenstate.groupby("participantId")["date"].nunique().reset_index(name="noOfDates")
dfb2 = screenstate.groupby("participantId")["date"].agg(["min", "max", "count"]).reset_index().rename(columns={'min': 'minDate', 'max': 'maxDate', 'count':'totalCount'})
dfb3 = pd.merge(dfb1, dfb2, on="participantId", how="inner")

# participantIds with more than 14days of data
dfb3 = dfb3[dfb3["noOfDates"]>=14].reset_index(drop=True)

# dfb3

In [5]:
# only 1 participant data exists in both backupfrigg1 and backupfrigg2
pd.merge(dfa3, dfb3, on="participantId", how="inner")

,participantId,noOfDates_x,minDate_x,maxDate_x,totalCount_x,noOfDates_y,minDate_y,maxDate_y,totalCount_y
0,PROSITC1705,65,2021-02-25,2021-05-08,3165,16,2021-05-14,2021-05-31,60


In [79]:
print(f"***participants in backup_frigg1:{dfa3.shape[0]},  participants in backup:{dfb3.shape[0]}, total Participants:{dfa3.shape[0]+dfb3.shape[0]}")

***participants in backup_frigg1:438,  participants in backup:85, total Participants:523


In [77]:
# participants with more that 14days of data
df3 = pd.concat([dfa3, dfb3], axis=0).reset_index(drop=True)
df3["participantId"] = df3["participantId"].str.replace("PROSITC000", "PROSITC")
df3["participantId"] = df3["participantId"].str.replace("PROSITC00", "PROSITC")
df3["participantId"] = df3["participantId"].str.replace("PROSITC0", "PROSITC")

df3.sort_values("participantId", inplace=True, ignore_index=True)
df3

,participantId,noOfDates,minDate,maxDate,totalCount
0,PROSITC10,36,2020-06-19,2020-07-31,4261
1,PROSITC100,33,2020-06-24,2020-07-27,2768
2,PROSITC1005,31,2020-12-25,2021-01-29,2944
3,PROSITC101,26,2020-07-02,2020-07-27,94
4,PROSITC1014,29,2020-12-22,2021-01-19,3717
5,PROSITC1018,29,2021-01-04,2021-02-01,3470
6,PROSITC102,42,2020-06-24,2020-08-06,3624
7,PROSITC1025,31,2021-01-10,2021-02-09,4241
8,PROSITC1026,34,2021-01-10,2021-02-13,2445
9,PROSITC1027,33,2021-01-10,2021-02-11,2574


In [8]:
# df3.loc[df3["participantId"].str.contains("PROSITC0"), :]

# df3["participantId"].str.replace("PROSITC0", "Test")
# df3["participantId"]
# df3.loc[df3["participantId"].str.contains("154"), :]

In [ ]:
# list of participants from backup and backup_frigg1 who have more that 14days of data

df3.participant.tolist()

In [9]:
# Personality scores
# scores = pd.read_csv("/csv/personality.csv")
# scores = scores[scores.record_id.str.contains("PROSITC")]

# labels
# file has personality score and diagnosis labels
labels = pd.read_csv("/csv/labels")
labels = labels[labels["participantId"].str.contains("PROSITC")]

In [10]:
labels

,participantId,con,agree,extra,neuro,open,age,gender,phone,dass,current_diagnosis
1,PROSITC001,6.0,7.0,5.0,10.0,6.0,16.0,0.0,1.0,0.0,0.0
2,PROSITC002,7.0,7.0,4.0,8.0,10.0,16.0,0.0,1.0,2.0,0.0
3,PROSITC003,10.0,10.0,6.0,9.0,9.0,20.0,0.0,1.0,3.0,0.0
4,PROSITC004,8.0,6.0,7.0,6.0,6.0,21.0,0.0,1.0,3.0,0.0
5,PROSITC005,8.0,6.0,7.0,6.0,6.0,20.0,0.0,1.0,3.0,0.0
6,PROSITC006,8.0,8.0,6.0,8.0,6.0,25.0,0.0,1.0,2.0,0.0
7,PROSITC007,8.0,6.0,7.0,3.0,8.0,23.0,1.0,1.0,2.0,0.0
8,PROSITC008,5.0,9.0,3.0,8.0,6.0,17.0,0.0,1.0,3.0,0.0
9,PROSITC009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,PROSITC010,8.0,7.0,3.0,10.0,7.0,20.0,0.0,1.0,1.0,0.0


In [12]:
labels = labels[~labels["dass"].isnull()]

In [14]:
labels["participantId"].tolist()

['PROSITC001',
 'PROSITC002',
 'PROSITC003',
 'PROSITC004',
 'PROSITC005',
 'PROSITC006',
 'PROSITC007',
 'PROSITC008',
 'PROSITC010',
 'PROSITC011',
 'PROSITC012',
 'PROSITC013',
 'PROSITC014',
 'PROSITC016',
 'PROSITC017',
 'PROSITC019',
 'PROSITC021',
 'PROSITC022',
 'PROSITC023',
 'PROSITC026',
 'PROSITC028',
 'PROSITC029',
 'PROSITC030',
 'PROSITC031',
 'PROSITC032',
 'PROSITC033',
 'PROSITC035',
 'PROSITC036',
 'PROSITC037',
 'PROSITC038',
 'PROSITC039',
 'PROSITC040',
 'PROSITC041',
 'PROSITC042',
 'PROSITC043',
 'PROSITC045',
 'PROSITC046',
 'PROSITC047',
 'PROSITC048',
 'PROSITC050',
 'PROSITC051',
 'PROSITC052',
 'PROSITC053',
 'PROSITC055',
 'PROSITC056',
 'PROSITC057',
 'PROSITC058',
 'PROSITC059',
 'PROSITC060',
 'PROSITC061',
 'PROSITC062',
 'PROSITC063',
 'PROSITC064',
 'PROSITC065',
 'PROSITC066',
 'PROSITC067',
 'PROSITC070',
 'PROSITC072',
 'PROSITC073',
 'PROSITC074',
 'PROSITC075',
 'PROSITC078',
 'PROSITC079',
 'PROSITC081',
 'PROSITC082',
 'PROSITC083',
 'PROSITC0

In [16]:
df_temp = pd.merge(df3, labels, on="participantId", how="inner")
df_temp

,participantId,noOfDates,minDate,maxDate,totalCount,con,agree,extra,neuro,open,age,gender,phone,dass,current_diagnosis
0,PROSITC003,31,2020-06-17,2020-07-18,1044,10.0,10.0,6.0,9.0,9.0,20.0,0.0,1.0,3.0,0.0
1,PROSITC007,29,2020-06-17,2020-07-16,5903,8.0,6.0,7.0,3.0,8.0,23.0,1.0,1.0,2.0,0.0
2,PROSITC008,31,2020-06-18,2020-07-20,537,5.0,9.0,3.0,8.0,6.0,17.0,0.0,1.0,3.0,0.0
3,PROSITC010,36,2020-06-19,2020-07-31,4261,8.0,7.0,3.0,10.0,7.0,20.0,0.0,1.0,1.0,0.0
4,PROSITC016,14,2020-06-18,2020-07-01,2805,7.0,8.0,5.0,9.0,4.0,17.0,0.0,1.0,2.0,0.0
5,PROSITC019,37,2020-06-19,2020-07-25,6697,10.0,9.0,6.0,10.0,6.0,21.0,0.0,1.0,3.0,0.0
6,PROSITC022,35,2020-06-17,2020-07-27,5740,8.0,9.0,6.0,6.0,10.0,23.0,0.0,1.0,1.0,0.0
7,PROSITC030,30,2020-06-21,2020-07-20,3378,10.0,10.0,6.0,5.0,6.0,20.0,1.0,1.0,2.0,0.0
8,PROSITC037,33,2020-06-20,2020-07-22,3291,7.0,7.0,4.0,9.0,8.0,25.0,0.0,1.0,3.0,0.0
9,PROSITC041,29,2020-06-18,2020-07-16,8092,9.0,4.0,2.0,9.0,8.0,22.0,0.0,1.0,3.0,0.0


In [26]:
# out of 503 participants only 231 participants have labels

df3.shape, labels.shape, df_temp.shape

((523, 5), (2485, 11), (477, 15))

In [42]:
set(df3.participantId) - set(labels.participantId)

{'PROSITC00154',
 'PROSITC00181',
 'PROSITC00185',
 'PROSITC00189',
 'PROSITC00190',
 'PROSITC00191',
 'PROSITC00195',
 'PROSITC00219',
 'PROSITC00226',
 'PROSITC00234',
 'PROSITC0239',
 'PROSITC1047',
 'PROSITC1742',
 'PROSITC1759',
 'PROSITC1826',
 'PROSITC2002',
 'PROSITC3111',
 'PROSITC3124',
 'PROSITC3144',
 'PROSITC3169',
 'PROSITC3172',
 'PROSITC3196',
 'PROSITC3201',
 'PROSITC3208',
 'PROSITC3213',
 'PROSITC3214',
 'PROSITC3217',
 'PROSITC3219',
 'PROSITC3221',
 'PROSITC3226',
 'PROSITC3227',
 'PROSITC3229',
 'PROSITC3230',
 'PROSITC3233',
 'PROSITC3234',
 'PROSITC3235',
 'PROSITC3244',
 'PROSITC3249',
 'PROSITC3259',
 'PROSITC3261',
 'PROSITC3279',
 'PROSITC3282',
 'PROSITC517',
 'PROSITC574',
 'PROSITC583',
 'PROSITC742'}

In [71]:
# l = labels.participantId.str.replace("PROSITC", "").astype('int').tolist()
# l = labels.participantId.str.replace("PROSITC0", "PROSITC").sort_values().tolist()
# sorted(l)
labels[labels.participantId.str.contains("154")]

,participantId,con,agree,extra,neuro,open,age,gender,phone,dass,current_diagnosis
127,PROSITC154,9.0,9.0,4.0,9.0,5.0,23.0,0.0,1.0,1.0,0.0
944,PROSITC1154,8.0,9.0,9.0,4.0,7.0,47.0,0.0,0.0,3.0,0.0
1270,PROSITC1540,10.0,6.0,5.0,8.0,7.0,39.0,0.0,0.0,2.0,0.0
1271,PROSITC1541,7.0,9.0,5.0,6.0,10.0,42.0,0.0,0.0,1.0,1.0
1272,PROSITC1542,5.0,10.0,8.0,8.0,6.0,34.0,0.0,0.0,1.0,0.0
1273,PROSITC1543,10.0,9.0,4.0,3.0,7.0,36.0,0.0,1.0,3.0,0.0
1274,PROSITC1544,7.0,8.0,6.0,5.0,9.0,19.0,1.0,1.0,3.0,0.0
1275,PROSITC1545,5.0,7.0,6.0,9.0,7.0,20.0,0.0,1.0,1.0,0.0
1276,PROSITC1547,7.0,2.0,6.0,10.0,6.0,64.0,0.0,1.0,0.0,0.0
1277,PROSITC1548,7.0,6.0,10.0,6.0,7.0,20.0,0.0,1.0,3.0,0.0


In [65]:
l1 = df3.participantId.sort_values(ascending=True).to_list()
sorted(l1)

['PROSITC00154',
 'PROSITC00181',
 'PROSITC00185',
 'PROSITC00189',
 'PROSITC00190',
 'PROSITC00191',
 'PROSITC00195',
 'PROSITC00219',
 'PROSITC00226',
 'PROSITC00234',
 'PROSITC003',
 'PROSITC007',
 'PROSITC008',
 'PROSITC010',
 'PROSITC016',
 'PROSITC019',
 'PROSITC022',
 'PROSITC0239',
 'PROSITC030',
 'PROSITC037',
 'PROSITC041',
 'PROSITC043',
 'PROSITC048',
 'PROSITC052',
 'PROSITC055',
 'PROSITC057',
 'PROSITC059',
 'PROSITC061',
 'PROSITC063',
 'PROSITC065',
 'PROSITC067',
 'PROSITC072',
 'PROSITC074',
 'PROSITC078',
 'PROSITC083',
 'PROSITC086',
 'PROSITC087',
 'PROSITC088',
 'PROSITC090',
 'PROSITC094',
 'PROSITC096',
 'PROSITC100',
 'PROSITC1005',
 'PROSITC101',
 'PROSITC1014',
 'PROSITC1018',
 'PROSITC102',
 'PROSITC1025',
 'PROSITC1026',
 'PROSITC1027',
 'PROSITC1029',
 'PROSITC1030',
 'PROSITC1031',
 'PROSITC1033',
 'PROSITC1034',
 'PROSITC1036',
 'PROSITC1037',
 'PROSITC1038',
 'PROSITC1039',
 'PROSITC1040',
 'PROSITC1041',
 'PROSITC1043',
 'PROSITC1044',
 'PROSITC1046',

In [64]:
set(l) - set(l1)

{1,
 2,
 4,
 5,
 6,
 11,
 12,
 13,
 14,
 17,
 21,
 23,
 26,
 28,
 29,
 31,
 32,
 33,
 35,
 36,
 38,
 39,
 40,
 42,
 45,
 46,
 47,
 50,
 51,
 53,
 56,
 58,
 60,
 62,
 64,
 66,
 70,
 73,
 75,
 79,
 81,
 82,
 84,
 85,
 92,
 93,
 95,
 99,
 103,
 104,
 105,
 107,
 112,
 113,
 114,
 116,
 118,
 119,
 120,
 123,
 125,
 126,
 128,
 129,
 130,
 131,
 132,
 134,
 136,
 139,
 140,
 142,
 143,
 144,
 145,
 147,
 149,
 151,
 152,
 153,
 156,
 158,
 160,
 161,
 162,
 163,
 165,
 166,
 167,
 168,
 169,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 186,
 187,
 188,
 192,
 193,
 194,
 196,
 198,
 199,
 200,
 201,
 202,
 203,
 206,
 207,
 210,
 211,
 212,
 213,
 214,
 216,
 217,
 218,
 221,
 223,
 225,
 227,
 229,
 231,
 233,
 235,
 237,
 240,
 241,
 242,
 245,
 246,
 248,
 249,
 250,
 251,
 252,
 254,
 257,
 259,
 260,
 262,
 263,
 264,
 265,
 267,
 269,
 271,
 272,
 273,
 275,
 276,
 279,
 280,
 281,
 283,
 284,
 286,
 287,
 288,
 289,
 290,
 293,
 295,
 296,
 298,
 299,
 301,
 30

In [ ]:
df_temp.to_csv("/csv/finalParticipants.csv", header=True, index=False)

In [18]:
df1 = pd.read_csv(os.path.join(dataPath1, "Lock_state.csv"), sep="|", header=None)

In [23]:
df1.iloc[:, 1].nunique()
# df1.columns

831